# __Predicting cell biological response__

In [ ]:
import wandb
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import logging

#### Run other .ipybn files

In [ ]:
# %run NN_model.ipynb
from NN_model import *

%run NN_dataset.ipynb

#### Dataset representation for NN model

In [ ]:
class CellDataset(Dataset):
    def __init__(self, data: pd.DataFrame, normalize: bool):
        # Save predictors as DataFrame
        self.cell_descriptors = data.drop(columns=['target'])

        # Normalize target value to [-1, 1]
        res = data['target']
        if normalize:
            res = res.apply(lambda x: -1 if x == 0 else 1)

        # Save target as DataFrame
        self.cell_response = res.astype('float64').to_frame()

    def __len__(self):
        return len(self.cell_descriptors)

    def __getitem__(self, idx):
        desc = self.cell_descriptors.iloc[idx]
        res = self.cell_response.iloc[idx]
        return desc.values, res.values
    
    def get_input_size(self):
        return self.cell_descriptors.shape[1]

## Wandb Project

In [ ]:
wandb.login()

%env "WANDB_NOTEBOOK_NAME" "NN-z1"

run = wandb.init(
    entity = "matus13579",  #dont change
    project = "NN-z1",      #dont change
    name = "test_run"       #run name
    #id =                   #define run with ID (used for resuming)
    #resume = True           #resume run
    )

#show graphs in Jupyter Notebook
#%%wandb                   

logger = logging.getLogger("wandb")
logger.setLevel(logging.ERROR)

## Experiments

In [ ]:
data = pd.read_csv('bioresponse.csv')

#### Load Dataset and Initialize NN model

In [ ]:
def setup_experiment(current_data):
    # Split dataset -> 20% testing, 80% training
    # Stratified split = each dataset has equal amounts of each class (saved in column 'target')
    train, test = train_test_split(current_data, test_size = 0.2, stratify = current_data['target']) 

    # Initlize dataset for NN
    train_data = CellDataset(train, normalize_dataset())
    test_data = CellDataset(test, normalize_dataset())

    # Create NN and training class
    mlp = MultiLayerPerceptron(train_data.get_input_size())
    trainer = Trainer(config, mlp)

    # Load dataset
    trainer.load_dataset(train_data, test_data)

    return trainer

### Training loop

In [ ]:
def run_experiment(myrun, trainer: Trainer, no_epochs):
    best_model = None
    best_accuracy = None

    for epoch in range(no_epochs):
        print (f"Epoch {epoch}")

        # Train model
        trainer.train()

        # Get metrics
        metrics = trainer.evaluate()

        loss_tr, loss_val = trainer.mean_loss()

        print (f"loss_training: {loss_tr} | loss_validate: {loss_val}")

        myrun.log({"loss_training": loss_tr})
        myrun.log({"loss_validate": loss_val})
        myrun.log({"accuracy": metrics.accuracy})
        myrun.log({"f1_score": metrics.accuracy})

        if (best_accuracy is None) or (best_accuracy > metrics.accuracy):
            best_accuracy = metrics.accuracy
            best_model = trainer.model


#### Experiment 1

In [ ]:
current_data = correlation_selection_merged(data)

config.activation_fn = "sigmoid"

In [ ]:
current_trainer = setup_experiment(current_data)

In [ ]:
current_trainer = setup_experiment(current_data)
run_experiment(run, current_trainer, 10)